# Ceneo Scraper

## Biblioteki

In [ ]:
import requests
from bs4 import BeautifulSoup

## Wysłanie do serwera żądania dostępu do zasobu

In [ ]:
product_id = "156802519"
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"
response = requests.get(url)
response.status_code

In [ ]:
page_dom = BeautifulSoup(response.text, "html.parser")
print(type(page_dom))
opinions = page_dom.select("div.js_product-review")
print(type(opinions))
opinion = page_dom.select_one("div.js_product-review")
print(type(opinion))
print(opinion)

## Ekstrakcja składowych pojedynczej opinii

|Składowa|Selektor|Zmienna|
|--------|--------|--------|
|id opinii|["data-entry-id"]|opinion_id|
|autor|span.user-post__author-name|author|
|rekomendacja|span.user-post__author-recomendation > em|recommendation|
|gwiazdki|span.user-post__score-count|rating|
|treść|div.user-post__text|content|
|lista zalet|div.review-feature__title--positives ~ div.review-feature__item|pros|
|lista wad|div.review-feature__title--negatives ~ div.review-feature__item|cons|
|dla ilu przydatna|button.vote-yes > span|useful|
|dla ilu nieprzydatna|button.vote-no > span|useless|
|data wystawienia|span.user-post__published > time:nth-child(1)["datetime"]|publish_date|
|data zakupu||purchase_date|

In [ ]:
opinion_id = opinion["data-entry-id"]
author = opinion.select_one(".user-post__author-name")
recommendation = opinion.select_one(".user-post__author-recomendation > em")
rating = opinion.select_one(".user-post__score-count")
content = opinion
pros = opinion
cons = opinion
useful = opinion
useless = opinion
publish_date = opinion
purchase_date = opinion
print(opinion_id)
print(author)
print(recommendation)
print(rating)